<a href="https://colab.research.google.com/github/nebyu08/data_sc/blob/main/kaggle_checking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers -q

In [2]:
!pip install accelerate -q -U

In [3]:
import accelerate
accelerator=accelerate.Accelerator()

In [4]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

from transformers import (AutoModel,AutoModelForMaskedLM,
                          AutoTokenizer, LineByLineTextDataset,
                          DataCollatorForLanguageModeling,
                          Trainer, TrainingArguments)

In [20]:
from huggingface_hub import notebook_login
notebook_login()

In [6]:
full=pd.read_csv('/content/total.csv')

In [7]:
from sklearn.model_selection import train_test_split

# Split the dataset into 80% training and 20% testing
train, test = train_test_split(full, test_size=0.2)

# Save the resulting DataFrames into CSV files
train.to_csv("train.csv", index=False)
test.to_csv("test.csv", index=False)

In [8]:
full.head()

,story,label
0,"\n\nOnce upon a time, there was a young man na...",depression
1,"and anxiety\n\nOnce upon a time, there was a ...",depression
2,"\n\nThe sun had risen over the horizon, but it...",depression
3,"and anxiety\n\nOnce upon a time, there was a ...",depression
4,"\n\nOnce upon a time, in a small village nestl...",depression


In [9]:
full['story'] = full['story'].apply(lambda x: x.replace('\n',''))

In [10]:
full.story[9]

" and anxietyOnce upon a time, there was a young woman named Sarah. She had always struggled with anxiety and depression, but it wasn't until she lost her job during the pandemic that it became unbearablevertically aligned textSarah's self-esteem plummeted, and she began to feel like she had no purpose in life. Every day was an endless struggle just to get out of bed and face the world.One day, Sarah decided to seek help. With the support of her therapist, she learned new coping strategies and began taking medication to help manage her symptoms. Slowly but surely, things started to improve. Sarah began feeling more hopeful and optimistic about the future."

In [11]:
model_name = "roberta-base"
tokenize_name="SamLowe/roberta-base-go_emotions"
model = AutoModelForMaskedLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(tokenize_name)
tokenizer.save_pretrained('./clrp_roberta_base');

In [12]:
train_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path='/content/train.csv', #mention train text file here
    block_size=256)

valid_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="/content/test.csv", #mention valid text file here
    block_size=256)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

training_args = TrainingArguments(
    output_dir="./clrp_roberta_base_chk", #select model path for checkpoint
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy= 'steps',
    save_total_limit=2,
    eval_steps=200,
    metric_for_best_model='eval_loss',
    greater_is_better=False,
    load_best_model_at_end =True,
    save_steps=400,
    prediction_loss_only=True,
    report_to = "none")

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset)

# training and saving the model

In [ ]:
print(model)

In [13]:
trainer.train()
#trainer.save_model(f'./clrp_roberta_base/psych_class')

Step,Training Loss,Validation Loss
200,No log,1.546045
400,No log,nan
600,1.209500,1.413344
800,1.209500,1.309380
1000,1.032300,1.355814
1200,1.032300,1.303528
1400,1.032300,1.242143
1600,0.917400,1.261644
1800,0.917400,nan
2000,0.827500,1.197677


There were missing keys in the checkpoint model loaded: ['lm_head.decoder.weight', 'lm_head.decoder.bias'].


In [14]:
!huggingface huggingface_hub --q

/bin/bash: line 1: huggingface: command not found


In [25]:
!transformers-cli repo create project-us

In [26]:
model.push_to_hub("project-us")

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/nebiyu29/project-us/commit/2c521fe60a681c707f09bf6603003734bcf6abfb', commit_message='Upload RobertaForMaskedLM', commit_description='', oid='2c521fe60a681c707f09bf6603003734bcf6abfb', pr_url=None, pr_revision=None, pr_num=None)

In [27]:
tokenizer.push_to_hub("project-us")

CommitInfo(commit_url='https://huggingface.co/nebiyu29/project-us/commit/7714ff6772192a7362051b0fcf6c5bf57d4d663a', commit_message='Upload tokenizer', commit_description='', oid='7714ff6772192a7362051b0fcf6c5bf57d4d663a', pr_url=None, pr_revision=None, pr_num=None)